In [1]:
import geopandas as gpd

gdf = gpd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\NLSDB\2025_03-03_NLSDB.gdb\CaseLands_03032025_1330.parquet")
gdf.shape

(1756089, 34)

In [6]:
import pandas as pd

df = pd.read_parquet(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.parquet")

In [5]:
df = pd.read_csv(r"C:\Users\es422\Documents\xentity\BLM\MLRS\Data\Snapshots\MLRS\2025-03-02_MLRS_Full\2025-03-02\consolidated_tables\CR_FULL_BLM_CASE.csv", sep='|')

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (3397559015.py, line 1)

In [ ]:
df.shape

In [8]:
df.shape

(6187848, 120)